In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torch import nn
# %load_ext cudf.pandas
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm.auto import tqdm
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
import torchmetrics
from torchvision import transforms
from torchmetrics import Accuracy
from transformers import AutoImageProcessor,AutoModel
try:
    from torchinfo import summary
except:
    !pip install torchinfo
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import warnings
warnings.filterwarnings("ignore")
import h5py
import io

2024-08-11 11:15:27.268534: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 11:15:27.268642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 11:15:27.395543: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov22/path_to_save_processor')
model = AutoModel.from_pretrained('/kaggle/input/dinov22/path_to_save_model')

In [3]:
class dino(nn.Module):
    def __init__(self,model):
        super().__init__()
        self.model=model
        self.block=nn.Linear(768,1)
        self.soft = nn.Sigmoid()
    def forward(self,x):
        x=self.model(x)
        x=self.block(x[1])
        return self.soft(x)

In [4]:
# import torch
# import torchvision.models as models
# import torch.nn as nn

# # Load the pretrained ResNet18 model
# resnet18 = models.resnet18(pretrained=True)

# # Replace the final fully connected layer
# num_features = resnet18.fc.in_features
# resnet18.fc = nn.Linear(num_features, 1)  # Binary classification output

# # Define the sigmoid layer
# sigmoid = nn.Sigmoid()

# # Create a custom model with ResNet18 followed by Sigmoid
# class ResNet18BinaryClassifier(nn.Module):
#     def __init__(self, base_model):
#         super(ResNet18BinaryClassifier, self).__init__()
#         self.base_model = base_model
#         self.sigmoid = nn.Sigmoid()
    
#     def forward(self, x):
#         x = self.base_model(x)
#         x = self.sigmoid(x)  # Apply sigmoid to output
#         return x

# # Initialize the custom model
# model = ResNet18BinaryClassifier(resnet18)

# # Set the model to evaluation mode
# # model.eval()

# Example of using the model for inference
# Assuming you have a sample input tensor `input_tensor`
# input_tensor = torch.randn(1, 3, 224, 224)  # Example input tensor

# output = model(input_tensor)

In [5]:
for param in model.parameters():
    param.requires_grad=False

In [6]:
df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv',low_memory=False)
image_data = h5py.File('/kaggle/input/isic-2024-challenge/train-image.hdf5' , 'r')
df['path'] = list(image_data.keys())

In [7]:
def seed_everything(seed=77):
#     random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()

In [8]:
t_1 = df[df['target']==1]
t_0 = df[df['target']==0].sample(len(t_1) * 10)
undersampled_df = pd.concat([t_1,t_0],axis=0).reset_index(drop=True)

In [9]:
class dataset(Dataset):
    def __init__(self,df,transformation):
        super().__init__()
        self.transformation = transformation
        self.y = df['target']
        self.df = df
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        image = np.array(image_data[self.df['path'][idx]]) 
        image = np.array(Image.open(io.BytesIO(image)),dtype=np.float32)
        if self.transformation:
            img = self.transformation(image=image)['image']
        return {'img':img,'y':self.y[idx]}

In [10]:
transformation = A.Compose([
    A.Resize(224,224),
    A.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225]),
    ToTensorV2()
])
eval_transformation = A.Compose([
    A.Resize(224,224),
    A.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225]),
    ToTensorV2()
])
df_total = dataset(undersampled_df,transformation)

In [11]:
from sklearn.model_selection import train_test_split
# Perform train-test split
train,test = train_test_split(df_total, test_size=0.03, random_state=42,stratify=undersampled_df['target'])
train,val = train_test_split(train, test_size=0.07, random_state=42,shuffle=True)

In [12]:
len(test),len(train) , len(val)

(130, 3899, 294)

In [13]:
train_dataloader = DataLoader(train,batch_size=64,shuffle=True)
valid_dataloader = DataLoader(val,batch_size=64,shuffle=False)
test_dataloader = DataLoader(test,batch_size=64,shuffle=False)

In [14]:
model = dino(model)
summary(model,(64,3,224,224))

Layer (type:depth-idx)                                       Output Shape              Param #
dino                                                         [64, 1]                   --
├─Dinov2Model: 1-1                                           [64, 768]                 --
│    └─Dinov2Embeddings: 2-1                                 [64, 257, 768]            1,053,696
│    │    └─Dinov2PatchEmbeddings: 3-1                       [64, 256, 768]            (452,352)
│    │    └─Dropout: 3-2                                     [64, 257, 768]            --
│    └─Dinov2Encoder: 2-2                                    [64, 257, 768]            --
│    │    └─ModuleList: 3-3                                  --                        (85,072,896)
│    └─LayerNorm: 2-3                                        [64, 257, 768]            (1,536)
├─Linear: 1-2                                                [64, 1]                   769
├─Sigmoid: 1-3                                               [64,

In [15]:
import numpy as np
import pandas as pd
import pandas.api.types
from sklearn.metrics import roc_curve, auc, roc_auc_score


def score(solution: pd.DataFrame, submission: pd.DataFrame, min_tpr: float=0.80) -> float:
    v_gt = abs(np.asarray(solution.values)-1)
    
    # flip the submissions to their compliments
    v_pred = -1.0*np.asarray(submission.values)

    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)
        
    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)
    
    return(partial_auc)

In [16]:
def train_model(model, epochs, train_dataloader, test_dataloader, loss_fn, optimizer):
    model = model.to(device)
    
    for epoch in tqdm(range(epochs), desc='Training...'):
        # Training phase
        model.train()
        train_loss = 0
        l = []
        s = []
        for data in tqdm(train_dataloader, desc=f'Epoch {epoch+1}/{epochs}', leave=False):
            X,y = data['img'],data['y']
            X, y = X.to(device).float(), y.to(device).long()  # Ensure labels are long for classification
            optimizer.zero_grad()
            y_logits = model(X)
            l.append(y_logits)
            s.append(y)
            loss = loss_fn(y_logits.squeeze().float(), y.float())
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_dataloader)
        f3=torch.cat(l)
        f4 = torch.cat(s)
        sol = pd.Series(f4.detach().to('cpu').squeeze().numpy())
        sub = pd.Series(f3.detach().to('cpu').squeeze().numpy()).astype(float)
        train_acc = score(sol,sub)
        
        # Evaluation phase
        model.eval()
        test_loss = 0
        l2 = []
        s2= []
        with torch.inference_mode():
            for data in tqdm(test_dataloader, desc='Evaluating', leave=False):
                X,y = data['img'],data['y']
                X, y = X.to(device).float(), y.to(device).long()  # Ensure labels are long for classification
                test_logits = model(X)
                l2.append(test_logits)
                s2.append(y)
                loss = loss_fn(test_logits.squeeze().float(), y.float())
                
                test_loss += loss.item()
                
        test_loss /= len(test_dataloader)
        f2 = torch.cat(l2)
        f1 = torch.cat(s2)
        sol = pd.Series(f1.detach().to('cpu').squeeze().numpy())
        sub = pd.Series(f2.detach().to('cpu').squeeze().numpy()).astype(float)
        test_acc = score(sol,sub)
        
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f} | Train pAUC: {train_acc:.4f}")
        print(f"Test Loss: {test_loss:.4f} | Test pAUC: {test_acc:.4f}")

In [17]:
loss_fn=nn.BCELoss()
optimizer=torch.optim.NAdam(model.parameters(),lr=0.003)

In [18]:
train_model(model,100,train_dataloader,valid_dataloader,loss_fn,optimizer)

Training...:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 1/100:   0%|          | 0/61 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/100
Train Loss: 0.2512 | Train pAUC: 0.0916
Test Loss: 0.2135 | Test pAUC: 0.1303


Epoch 2/100:   0%|          | 0/61 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 99/100
Train Loss: 0.0731 | Train pAUC: 0.1921
Test Loss: 0.2929 | Test pAUC: 0.1135


Epoch 100/100:   0%|          | 0/61 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 100/100
Train Loss: 0.0738 | Train pAUC: 0.1916
Test Loss: 0.2593 | Test pAUC: 0.1175


In [19]:
model.eval()
test_loss = 0
l2 = []
s2= []
with torch.inference_mode():
    for data in tqdm(test_dataloader, desc='Evaluating', leave=False):
        X,y = data['img'],data['y']
        X, y = X.to(device).float(), y.to(device).long()  # Ensure labels are long for classification
        test_logits = model(X)
        l2.append(test_logits)
        s2.append(y)
        loss = loss_fn(test_logits.squeeze().float(), y.float())
                
        test_loss += loss.item()
                
    test_loss /= len(test_dataloader)
    f2 = torch.cat(l2)
    f1 = torch.cat(s2)
    sol = pd.Series(f1.detach().to('cpu').squeeze().numpy())
    sub = pd.Series(f2.detach().to('cpu').squeeze().numpy()).astype(float)
test_acc = score(sol,sub)
print('most probably the LB score : ',test_acc)

Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

most probably the LB score :  0.17810734463276834


In [20]:
df_test = pd.read_csv('/kaggle/input/isic-2024-challenge/test-metadata.csv',low_memory=False)
image_data = h5py.File('/kaggle/input/isic-2024-challenge/test-image.hdf5' , 'r')
test_grp = list(image_data.keys())
class testdataset(Dataset):
    def __init__(self,transformation):
        super().__init__()
        self.transformation = transformation
    def __len__(self):
        return len(df_test)
    def __getitem__(self,idx):
        image = np.array(image_data[test_grp[idx]]) 
        image = np.array(Image.open(io.BytesIO(image)),dtype=np.float32)
        if self.transformation:
            img = self.transformation(image=image)['image']
        return {'img':img}
eval_transformation = A.Compose([
    A.Resize(180,180),
    A.Normalize(mean = [0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225]),
    ToTensorV2()
])
df_test_dataset = testdataset(eval_transformation)
df_test_dataloader = DataLoader(df_test_dataset,batch_size=64,shuffle=False)

In [21]:
sub = pd.read_csv('/kaggle/input/isic-2024-challenge/sample_submission.csv',index_col='isic_id')
model.eval()
model = model.to(device)
l2 = []
with torch.inference_mode():
    for data in tqdm(df_test_dataloader, desc='Evaluating', leave=False):
        X= data['img']
        X= X.to(device).float()
        test_logits = model(X)
        l2.append(test_logits.to('cpu'))
f2 = torch.cat(l2)
sub['target'] = [i.item() for i in f2]
sub

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

,target
isic_id,
ISIC_0015657,0.000775
ISIC_0015729,0.000002
ISIC_0015740,0.655491


In [22]:
submission = sub.to_csv('/kaggle/working/submission.csv')